Follows:  https://www.youtube.com/watch?v=5ypQIUbpA7c

In [16]:
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
from PIL import Image
import torch
from transformers import GLPNImageProcessor, GLPNForDepthEstimation
import numpy as np
import open3d as o3d

# Model

In [8]:
feature_extractor = GLPNImageProcessor.from_pretrained("vinvino02/glpn-nyu")
model = GLPNForDepthEstimation.from_pretrained("vinvino02/glpn-nyu")

preprocessor_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/920 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/245M [00:00<?, ?B/s]

# Load and Resize Image

In [11]:
image = Image.open("/Users/emeronmarcelle/Downloads/IMG_8363.jpg")
new_height = 480 if image.height > 480 else image.height
new_width = int(new_height * image.width / image.height)
diff = new_width % 32

new_width = new_width - diff if diff < 16 else new_width + 32 - dff
new_size = (new_width, new_height)
image = image.resize(new_size)


# Prepare the Image

In [12]:
inputs = feature_extractor(images= image, return_tensors='pt')

# Get prediction from Model

In [13]:
with torch.no_grad():
    outputs = model(**inputs)
    predicted_depth = outputs.predicted_depth

# Post-Processing

In [14]:
pad = 16
output = predicted_depth.squeeze().cpu().numpy() * 1000.0
output = output[pad:-pad, pad:-pad]
image = image.crop((pad, pad, image.width - pad, image.height - pad))

# Visualize the prediction

In [45]:
fig, ax = plt.subplots(1,2)
ax[0].imshow(image)
ax[0].tick_params(left=False, bottom=False, labelleft=False, labelbottom= False)
ax[1].imshow(output, cmap='plasma')
ax[1].tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)
plt.tight_layout()
plt.pause(5)

# Prepare the depth image for open3d

In [17]:
width, height = image.size
depth_image = (output * 255 / np.max(output)).astype('uint8')
image = np.array(image)

In [18]:
#create rgbd image
depth_o3d = o3d.geometry.Image(depth_image)
image_o3d = o3d.geometry.Image(image)
rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(image_o3d,depth_o3d,
                                                                convert_rgb_to_intensity=False)


# Creating Camera

In [19]:
camera_intrisic = o3d.camera.PinholeCameraIntrinsic()
camera_intrisic.set_intrinsics(width, height, 500, 500, width/2, height/2)

# Creating o3d point cloud

In [46]:
pcd_raw = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd_image,camera_intrisic)
#Visualize the 3d image
o3d.visualization.draw_geometries([pcd_raw])

# Post-processing the 3D Point Cloud

In [47]:
#outlier removal
#adjust std_ratio for better performance
cl, ind = pcd_raw.remove_statistical_outlier(nb_neighbors=20, std_ratio=20.0)
pcd = pcd_raw.select_by_index(ind)

#estiamte normals
pcd.estimate_normals()
pcd.orient_normals_to_align_with_direction()
#Rerun visualization with outlier removal
o3d.visualization.draw_geometries([pcd])

# Surface reconstruction

In [48]:
#adjust depth volumne as needed
mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=10, n_threads=1)[0]

# rotate the mesh
rotation = mesh.get_rotation_matrix_from_xyz((np.pi, 0, 0))
mesh.rotate(rotation, center=(0, 0, 0))

# visualize the mesh
o3d.visualization.draw_geometries([mesh], mesh_show_back_face=True)

In [49]:
mesh_uniform = mesh.paint_uniform_color([0.9, 0.8, 0.9])
mesh_uniform.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh_uniform], mesh_show_back_face=True)

# Export 3D Mesh

In [44]:
#format is .obj or .ply
o3d.io.write_triangle_mesh('portrait.ply', mesh)

True